In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

def validate_crypto_symbol(symbol):
    """动态验证交易对有效性"""
    # 构造标准交易对格式
    formatted_symbol = f"t{symbol.upper()}USD"
    
    # 发送测试请求（获取最近1小时数据）
    test_url = f'https://api-pub.bitfinex.com/v2/candles/trade:1h:{formatted_symbol}/hist'
    params = {
        "limit": 1,
        "sort": -1
    }
    
    try:
        response = requests.get(test_url, params=params, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data and isinstance(data, list) and len(data[0]) >= 6:
                return formatted_symbol  # 返回验证后的正式交易对
        return None
    except requests.exceptions.RequestException:
        return None

def get_valid_symbol():
    """获取并验证用户输入的加密货币"""
    while True:
        user_input = input("请输入加密货币代号（例如BTC）：").strip().upper()
        
        # 退出机制
        if user_input.lower() == 'exit':
            print("程序已退出")
            exit()
            
        if not user_input.isalpha():
            print("错误：代号只能包含字母")
            continue
            
        # 动态验证
        valid_symbol = validate_crypto_symbol(user_input)
        if valid_symbol:
            return user_input.upper(), valid_symbol
            
        print(f"错误：{user_input} 不是有效的交易对或API不可用")
        print("提示：")
        print("- 请确认代号的正确性（如BTC、ETH等）")
        print("- 确保代号的完整形式（如XBT应为BTC）")
        print("- 输入'exit'退出程序")

def fetch_crypto_data(symbol, valid_symbol):
    """获取指定加密货币数据"""
    print(f"\n正在获取 {symbol} 的过去7天每小时数据...")
    
    # 设置时间范围
    end_date = datetime.now()
    start_date = end_date - timedelta(days=7)
    
    # 转换为时间戳（精确到毫秒）
    start_ts = int(start_date.timestamp() * 1000)
    end_ts = int(end_date.timestamp() * 1000)
    
    # 构建请求
    url = f'https://api-pub.bitfinex.com/v2/candles/trade:1h:{valid_symbol}/hist'
    params = {
        "start": start_ts,
        "end": end_ts,
        "sort": 1,
        "limit": 1000  # 最大允许值
    }
    
    try:
        response = requests.get(url, params=params, timeout=15)
        response.raise_for_status()
        data = response.json()
        
        if not data:
            print("警告：获取到空数据集")
            return None
            
        # 处理数据
        df = pd.DataFrame(data, columns=["timestamp", "open", "close", "high", "low", "volume"])
        df['price_change_pct'] = (df['close'] - df['open']) / df['open'] * 100
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
        
        # 数据清洗
        df = df.dropna(subset=['open', 'close'])
        df = df.sort_values('datetime')
        
        # 列名处理
        df = df.rename(columns={
            "volume": f"{symbol}_volume",
            "timestamp": "original_timestamp"
        })
        
        return df[['datetime', 'open', 'high', 'low', 'close', 
                 f'{symbol}_volume', 'price_change_pct']]
        
    except requests.exceptions.HTTPError as e:
        print(f"API错误：{str(e)}")
        print(f"响应内容：{response.text[:200]}")
    except Exception as e:
        print(f"获取数据失败：{str(e)}")
    
    return None

# 主程序
if __name__ == "__main__":
    print("=== 加密货币数据采集器 ===")
    print("支持所有Bitfinex平台存在的加密货币")
    print("输入'exit'可随时退出程序\n")
    
    # 获取有效交易对
    symbol, valid_symbol = get_valid_symbol()
    
    # 获取数据
    df = fetch_crypto_data(symbol, valid_symbol)
    
    if df is not None and not df.empty:
        # 保存文件
        filename = f"../data/{symbol}_7d_hourly.csv"
        try:
            df.to_csv(filename, index=False)
            print(f"\n成功保存 {len(df)} 条数据到 {filename}")
            print("数据时间范围：")
            print(f"开始：{df['datetime'].min()}")
            print(f"结束：{df['datetime'].max()}")
            
            # 显示统计信息
            print("\n数据统计：")
            print(f"价格变化范围：{df['price_change_pct'].min():.2f}% 到 {df['price_change_pct'].max():.2f}%")
            print(f"平均交易量：{df[f'{symbol}_volume'].mean():.2f}")
        except Exception as e:
            print(f"保存文件失败：{str(e)}")
    else:
        print("\n无法获取有效数据，可能原因：")
        print("- 该交易对在指定时间段无数据")
        print("- 交易所API暂时不可用")
        print("- 网络连接问题")

=== 加密货币数据采集器 ===
支持所有Bitfinex平台存在的加密货币
输入'exit'可随时退出程序


请输入加密货币代号（例如BTC）： btc



正在获取 BTC 的过去7天每小时数据...

成功保存 168 条数据到 BTC_7d_hourly.csv
数据时间范围：
开始：2025-01-23 03:00:00+00:00
结束：2025-01-30 02:00:00+00:00

数据统计：
价格变化范围：-1.60% 到 2.63%
平均交易量：25.78
